1. Get galactic coordinates from ra and dec given in ```public_void_catalog.dat```
2. The comoving distance $d$ can be found then by doing $d = \sqrt{x^2 + y^2 + z^2}$ using the $x$, $y$ and $z$ given in the same catalog.

As a result we have all the coordinates for every void centre.

3. 

In [28]:
import os
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import QTable

cosmo = FlatLambdaCDM(H0 = 70, Om0 = 0.27, Tcmb0 = 2.725)

# importing void and galaxy data

data_gal = np.loadtxt('maglim_void_galaxies.txt')
data_void = np.loadtxt('public_void_catalog.txt')

# stripping the data

rdshft_gal = data_gal[:, 2] #redshift of void galaxies
#cmvdist_gal = data_gal[:, 3] #comoving distance of void galaxies

x_void = data_void[:, 2] 
y_void = data_void[:, 3]
z_void = data_void[:, 4]
void_radius = data_void[:, 5] 

x_gal = data_gal[:, 6]
y_gal = data_gal[:, 7]
z_gal = data_gal[:, 8]

#print(cmvdist_gal)

#following code to check if comoving distances are same as given in catalog
cmvdist_gal = np.sqrt(x_gal**2+y_gal**2+z_gal**2)*0.7
cmvdist_void  = np.sqrt(x_void**2+y_void**2+z_void**2)*0.7


#arr1 = cosmo.comoving_distance(rdshft_gal) #this does not give same distances as given in catalog

#print(arr1)

#following part does calculation of galactic coordinates but I ended up not using them

ra_void = data_void[:, 0]
dec_void = data_void[:, 1]

c_icrs = SkyCoord(ra=ra_void*u.degree, dec=dec_void*u.degree, frame='icrs')

galcoord_void = c_icrs.galactic

#following wrote the gal long and gal lat into a .ecsv file from where I then imported the values
t = QTable([galcoord_void], names=["skycoord"])
#t.write('galcoord_void.ecsv')

galcoord_void_data = np.loadtxt('galcoord_void.ecsv')

l_void = galcoord_void_data[:, 0]
b_void = galcoord_void_data[:, 1]



coord_void = []
for i in range(len(l_void)):
    coord1 = []
    coord1.append(cmvdist_void[i])
    coord1.append(l_void[i])
    coord1.append(b_void[i])
    coord_void.append(coord1)


#print(coord_void)






In [29]:
# same as above, but for galaxies

ra_gal = data_gal[:, 0]
dec_gal = data_gal[:, 1]

c_icrs1 = SkyCoord(ra=ra_gal*u.degree, dec=dec_gal*u.degree, frame='icrs')

galcoord_gal = c_icrs1.galactic


t = QTable([galcoord_gal], names=["skycoord"])
#t.write('galcoord_gal.ecsv')

In [30]:
galcoord_gal_data = np.loadtxt('galcoord_gal.ecsv')

l_gal = galcoord_gal_data[:, 0]
b_gal = galcoord_gal_data[:, 1]

coord_gal = []
for i in range(len(l_gal)):
    coord1 = []
    coord1.append(cmvdist_gal[i])
    coord1.append(l_gal[i])
    coord1.append(b_gal[i])
    coord_gal.append(coord1)




In [31]:
print(len(coord_void))
print(len(coord_gal))
print(cmvdist_gal)

1055
79947
[102.17899562 135.46399818 163.54800084 ... 149.24699907 172.41699532
 175.20999912]


Find distance of each void galaxy from each void centre (take a void centre and calculate its distance to every void galaxy, then move on to next void centre). Set a threshold/cut-off value here, disregarding all the galaxies, and a preliminary filter will be complete

In [ ]:
#this is the main code, nothing special, just uses separation_3d to find distance between 2 points. 

voidGalxies = []


for i in range(len(coord_void)):
    for j in range(len(coord_gal)):
        c1 = SkyCoord(ra=ra_gal[j]*u.degree, dec=dec_gal[j]*u.degree, distance=cmvdist_gal[j]*u.Mpc)
        c2 = SkyCoord(ra=ra_void[i]*u.degree, dec=dec_void[i]*u.degree, distance=cmvdist_void[i]*u.Mpc)
        dist1 = c1.separation_3d(c2)
        if dist1<=void_radius[i]*u.Mpc:
            voidGalxies.append((i+1, j, dist1, void_radius[i]))

print(voidGalxies)


In [39]:
voidGalxies = []
col_format = "{:<25}" * 2 + "\n"

for i in range(len(coord_void)):#range(len(coord_void)):
    c2 = SkyCoord(ra=ra_gal*u.degree, dec=dec_gal*u.degree, distance=cmvdist_gal*u.Mpc)
    c1 = SkyCoord(ra=ra_void[i]*u.degree, dec=dec_void[i]*u.degree, distance=cmvdist_void[i]*u.Mpc)
    dist = c1.separation_3d(c2)
    frac_dist = dist.value/void_radius[i]
    #print(dist)
    #print(len(dist))
    with open(os.path.join("/home/peakcipher/projects/t.ghosh/void_galaxies/void_data","void_Number{}.txt".format(i+1)), "w") as f:
        for j in zip(dist.value, frac_dist):
        #for j in range(len(dist)):
            f.write(col_format.format(*j))
            #f.write("%s\n" % dist.value[j])
    #dist1 would be of the size of Ngal
    #voidGalxies.append((i+1, j, dist1, void_radius[i]))
    #print(np.size(dist)==np.size(ra_gal))